In [2]:
import pandas as pd

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
df = pd.read_csv('/kaggle/input/filteredbooksreview/books_with_raw_review.csv')
df.head()

,Unnamed: 0,Id,categories,published_date,user_id,useful,total_of_person,review_score,review_text
0,1,0826414346,['Biography & Autobiography'],2005-01-01,A30TK6U7DNS82R,10,10,5.0,I don't care much for Dr. Seuss but after read...
1,2,0826414346,['Biography & Autobiography'],2005-01-01,A3UH4UZ4RSVO82,10,11,5.0,"If people become the books they read and if ""t..."
2,3,0826414346,['Biography & Autobiography'],2005-01-01,A2MVUWT453QH61,7,7,4.0,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
3,4,0826414346,['Biography & Autobiography'],2005-01-01,A22X4XUPKF66MR,3,3,4.0,Philip Nel - Dr. Seuss: American IconThis is b...
4,5,0826414346,['Biography & Autobiography'],2005-01-01,A2F6NONFUDB6UK,2,2,4.0,"""Dr. Seuss: American Icon"" by Philip Nel is a ..."


In [6]:
df.drop('Unnamed: 0', axis =1, inplace = True)

In [7]:
df.head()

,Id,categories,published_date,user_id,useful,total_of_person,review_score,review_text
0,0826414346,['Biography & Autobiography'],2005-01-01,A30TK6U7DNS82R,10,10,5.0,I don't care much for Dr. Seuss but after read...
1,0826414346,['Biography & Autobiography'],2005-01-01,A3UH4UZ4RSVO82,10,11,5.0,"If people become the books they read and if ""t..."
2,0826414346,['Biography & Autobiography'],2005-01-01,A2MVUWT453QH61,7,7,4.0,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
3,0826414346,['Biography & Autobiography'],2005-01-01,A22X4XUPKF66MR,3,3,4.0,Philip Nel - Dr. Seuss: American IconThis is b...
4,0826414346,['Biography & Autobiography'],2005-01-01,A2F6NONFUDB6UK,2,2,4.0,"""Dr. Seuss: American Icon"" by Philip Nel is a ..."


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from tqdm import tqdm
import torch

# # Set multiprocessing start method to 'spawn'
# mp.set_start_method('spawn', force=True)

# Load the pre-trained model and tokenizer
model_name = "bhadresh-savani/distilbert-base-uncased-emotion"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_emotion_bert(texts):
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        emotions = torch.argmax(logits, dim=1).tolist()
    return emotions

# # Convert tokens to string texts
# df['review_tokens'] = df['tokens'].apply(lambda tokens: " ".join(tokens))

# Set a smaller batch size to fit within memory constraints
batch_size = 32
emotions = []
# Initialize the 'emotions' column with None
df['emotions'] = None

for i in tqdm(range(0, len(df), batch_size)):
    batch_texts = df['review_text'][i:i+batch_size].tolist()
    batch_emotions = get_emotion_bert(batch_texts)
    emotions.extend(batch_emotions)

# Assign final emotions
df['emotions'] = emotions

# Save the final DataFrame
df.to_csv('/kaggle/working/emotion_classified_reviews.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pickle

# One-hot encode the emotions for the training and test sets
encoder = OneHotEncoder(sparse=False)
train_emotions_ohe = encoder.fit_transform(train_data[['emotions']])
test_emotions_ohe = encoder.transform(test_data[['emotions']])

# Convert the encoded emotions to DataFrames
train_emotions_df = pd.DataFrame(train_emotions_ohe, columns=[f"emotion_{i}" for i in range(train_emotions_ohe.shape[1])])
test_emotions_df = pd.DataFrame(test_emotions_ohe, columns=[f"emotion_{i}" for i in range(test_emotions_ohe.shape[1])])

# Concatenate the encoded emotions with the original data
train_data = pd.concat([train_data.reset_index(drop=True), train_emotions_df], axis=1)
test_data = pd.concat([test_data.reset_index(drop=True), test_emotions_df], axis=1)

# Drop the 'emotions' column if not needed
train_data.drop('emotions', axis=1, inplace=True)
test_data.drop('emotions', axis=1, inplace=True)

# Display the size of the splits and the first few rows
print(f"Training data size: {train_data.shape}")
print(f"Test data size: {test_data.shape}")
print(train_data.head())
print(test_data.head())

# Save encoder to a pkl file
with open("/kaggle/working/emotion_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

print("Encoder saved successfully.")

In [ ]:
train_data.to_csv('/kaggle/working/train_data.csv', index=False)
test_data.to_csv('/kaggle/working/test_data.csv', index=False)